In [ ]:
import sqlite3
import csv

# Connect to SQLite database
conn = sqlite3.connect('library11.db')
cursor = conn.cursor()

# Function to insert data into Books table
def insert_books_data(row):
    cursor.execute('''
            INSERT INTO Books (BK_ID, BK_NAME, AUTHOR_NAME, Edition, Date_Year, Publisher, Place, ISBN, Price, Pages, Category, Department, Subject) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['Access_No'], row['Title'], row['Authors'], row['Edition'], row['Date_Year'],
          row['Publisher'], row['Place'], row['ISBN'], row['Price'], row['Pages'],row['Category'],row['Department'],row['Subject']))

# Function to insert data into BookDetails table
def insert_book_details_data(row):
    cursor.execute('''
        INSERT INTO BookDetails (Asset_Code, Call_No, BK_ID)
        VALUES (?, ?, ?)
    ''', (row['Asset_Code'], row['Call_No'], row['Access_No']))


# Function to insert data into BookInventory table
def insert_book_inventory_data(row):
    cursor.execute('''
        INSERT INTO BookInventory (BK_ID, Number_of_Copies)
        VALUES (?, ?)
    ''', (row['Access_No'], row['copies']))


# Function to insert data into Transactions table
def insert_transactions_data(row):
    cursor.execute('''
        INSERT INTO Transactions (BK_ID, Bill_Date, Vendor)
        VALUES (?, ?, ?)
    ''', (row['Access_No'], row['Bill_Date'], row['Vendor']))

# Function to handle the CSV data insertion into respective tables
def process_csv_data():
    with open('C:/Users/Hunny/Downloads/updated_file.csv', newline='', encoding='latin-1') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Insert data into respective tables
            insert_books_data(row)
            insert_book_details_data(row)
            insert_book_inventory_data(row)
            insert_transactions_data(row)
            # Add other table insertions if necessary

    conn.commit()
    print("Data inserted successfully.")

# Call the function to insert data from CSV into respective tables
process_csv_data()

# Close connection
conn.close()


In [81]:
a =cursor.execute('PRAGMA table_info(BookDetails);')
print(a)

In [1]:
import sqlite3

# Connect to SQLite database (create if not exists)
conn = sqlite3.connect('library11.db')
cursor = conn.cursor()

# Drop existing tables if they exist
cursor.execute("DROP TABLE IF EXISTS BookInventory")
cursor.execute("DROP TABLE IF EXISTS Transactions")
cursor.execute("DROP TABLE IF EXISTS BookDetails")
cursor.execute("DROP TABLE IF EXISTS Books")

# Create Books table with merged fields for category, department, and subject
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Books (
        BK_ID TEXT PRIMARY KEY NOT NULL,
        BK_NAME TEXT,
        AUTHOR_NAME TEXT,
        Edition TEXT,
        Date_Year TEXT,
        Publisher TEXT,
        Place TEXT,
        ISBN TEXT,
        Price TEXT,
        Pages TEXT,
        Category TEXT,
        Department TEXT,
        Subject TEXT
    )
''')

# Create BookInventory table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS BookInventory (
        BK_ID TEXT PRIMARY KEY NOT NULL,
        Number_of_Copies INTEGER,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')

# Create BookDetails table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS BookDetails (
        Asset_Code TEXT,
        Call_No TEXT,
        BK_ID TEXT NOT NULL,
        BK_STATUS TEXT,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')

# Create Transactions table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Transactions (
        Transaction_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        BK_ID TEXT,
        CARD_ID TEXT,
        Bill_Date TEXT,
        Vendor TEXT,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')


# Create VIEW Library
cursor.execute('''
    CREATE VIEW IF NOT EXISTS Library AS
    SELECT
        Books.BK_NAME AS book_name,
        Books.BK_ID AS book_id,
        Books.AUTHOR_NAME AS author_name,
        BookDetails.BK_STATUS AS book_status,
        Transactions.CARD_ID AS card_id,
        Books.Category AS category,
        BookDetails.Asset_Code AS asset_code,
        Books.Department AS department,
        Books.Subject AS subject,
        Books.Edition AS edition,
        Books.Date_Year AS date_year,
        BookDetails.Call_No AS call_no, 
        Books.Publisher AS publisher,
        Books.Place AS place,
        Books.ISBN AS isbn,
        Books.Price AS price,
        Transactions.Bill_Date AS bill_date,
        Transactions.Vendor AS vendor,
        Books.Pages AS pages,
        BookInventory.Number_of_Copies AS num_copies
    FROM
        Books
    LEFT JOIN BookDetails ON Books.BK_ID = BookDetails.BK_ID
    LEFT JOIN Transactions ON Books.BK_ID = Transactions.BK_ID
    LEFT JOIN BookInventory ON Books.BK_ID = BookInventory.BK_ID
''')

print("Tables created successfully.")
conn.commit()
conn.close()


Tables created successfully.


In [1]:
import sqlite3 as lite
import pandas as pd

In [2]:
con = lite.connect('library11.db')

In [4]:
con.execute("UPDATE BookInventory SET Number_of_Copies = 1 WHERE Number_of_Copies IS NULL OR Number_of_Copies NOT BETWEEN 2 AND 100;")

In [12]:
con.execute("""CREATE TRIGGER prevent_negative_copies
BEFORE UPDATE ON BookInventory
FOR EACH ROW
WHEN NEW.Number_of_Copies < 0
BEGIN
    SELECT RAISE(ABORT, 'Cannot set Number_of_Copies below 0');
END;
""")

In [4]:
#for issueing
con.execute(""" 
            CREATE TRIGGER update_copies_on_issue
AFTER UPDATE ON Transactions
FOR EACH ROW
WHEN NEW.CARD_ID <> 'N/A' AND OLD.CARD_ID = 'N/A'
BEGIN
    UPDATE BookInventory
    SET Number_of_Copies = Number_of_Copies - 1
    WHERE BK_ID = NEW.BK_ID;
END;
""")

In [5]:
#for returning
con.execute(""" 
CREATE TRIGGER update_copies_on_return
AFTER UPDATE ON Transactions
FOR EACH ROW
WHEN NEW.CARD_ID = 'N/A' AND OLD.CARD_ID <> 'N/A'
BEGIN
    UPDATE BookInventory
    SET Number_of_Copies = Number_of_Copies + 1
    WHERE BK_ID = NEW.BK_ID;
END;            
""")

In [5]:
con.commit()

In [5]:
pd.read_sql("SELECT * FROM BookInventory;",con)

,BK_ID,Number_of_Copies
0,1,9
1,9,2
2,11,2
3,13,8
4,21,2
...,...,...
794,1097,1
795,1098,1
796,1099,1
797,1100,1


In [4]:
con.execute("UPDATE BookDetails SET BK_STATUS= 'Available' ")

In [7]:
con.close()